In [2]:
import cv2
import numpy as np

# Function to stabilize a video using template matching
def stabilize_video(input_video_path, output_video_path, template_path, threshold=0.8):
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)  # Load the template as grayscale
    if template is None:
        print("Error: Could not open template image.")
        return

    # Resize the template if it's larger than the video frames
    if template.shape[0] > cap.get(4) or template.shape[1] > cap.get(3):
        template = cv2.resize(template, (int(cap.get(3)), int(cap.get(4))))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

    prev_frame = None

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        if prev_frame is None:
            prev_frame = frame
            out.write(frame)
            continue

        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        current_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Perform template matching
        result = cv2.matchTemplate(current_gray, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

        if max_val >= threshold:
            # Calculate the translation vector
            x_offset = max_loc[0] - template.shape[1] // 2
            y_offset = max_loc[1] - template.shape[0] // 2

            # Apply the translation to stabilize the frame
            translation_matrix = np.float32([[1, 0, x_offset], [0, 1, y_offset]])
            stabilized_frame = cv2.warpAffine(frame, translation_matrix, (frame.shape[1], frame.shape[0]))

            out.write(stabilized_frame)
        else:
            out.write(frame)

        prev_frame = frame

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("Video stabilization complete.")

input_video_path = "videos/unstablized-video.mp4"
output_video_path = "videos/output_video.mp4"
template_path = "images/Vivek.png"
threshold = 0.8

stabilize_video(input_video_path, output_video_path, template_path, threshold)

Video stabilization complete.
